In [66]:
import collections
import itertools
from boltons.queueutils import HeapPriorityQueue
import networkx as nx
import networkx.algorithms.community.label_propagation as label_prop
import networkx.algorithms.community.modularity_max as modularity
import networkx.algorithms.components.connected as components

In [67]:
CLUSTERING_FN = {
    1: label_prop.label_propagation_communities,
    2: modularity.greedy_modularity_communities,
    3: components.connected_components
}


In [68]:
def CreateEgonets(graph):
    completed_nodes = set()
    
    ego_egonet_map = collections.defaultdict(nx.Graph)
    degrees_pq = HeapPriorityQueue()
    curr_degree = {}
    for node in graph.nodes:
        degrees_pq.add(node, -graph.degree[node])
        curr_degree[node] = graph.degree[node]
    edge_set = set(graph.edges)
    
    
    while degrees_pq:
        node = degrees_pq.pop()
#         print("---",node)
        for neighbor in graph.neighbors(node):
            if neighbor == node:
                continue
            ego_egonet_map[node].add_node(neighbor)  
#             print(ego_egonet_map)
            if neighbor not in completed_nodes:
                curr_degree[neighbor] -= 1
                degrees_pq.remove(neighbor)
                degrees_pq.add(neighbor, -curr_degree[neighbor])
        not_removed = []
#         print(ego_egonet_map)
        for neighbor in graph.neighbors(node):
            if neighbor not in completed_nodes:
                not_removed.append(neighbor)
        for pos_u, u in enumerate(not_removed):
            for v  in not_removed[pos_u+1:]:
                if (u, v) in edge_set or (v, u) in edge_set:
                    ego_egonet_map[node].add_edge(u, v)
                    ego_egonet_map[u].add_edge(node, v)
                    ego_egonet_map[v].add_edge(u,node)
        completed_nodes.add(node)
#         print(completed_nodes)
    with open("1.txt", 'w') as outfile:
        for egonet in ego_egonet_map:
            outfile.write(f"Egonet of {egonet} is \n {ego_egonet_map[egonet].edges()}\n\n\n")
#     print(ego_egonet_map)
    return ego_egonet_map

In [69]:
# import networkx as nx
# G= nx.read_edgelist('D:\wsc project\paper_dataset.csv',delimiter=",")
# CreateEgonets(G)


In [70]:
def CreatePersonaGraph(graph,clustering_fn=label_prop.label_propagation_communities,persona_start_id=0):
    egonets = CreateEgonets(graph)
    node_neighbor_persona_id_map = collections.defaultdict(dict)
    persona_graph = nx.Graph()
    persona_to_original_mapping = dict()
    persona_id_counter = itertools.count(start=persona_start_id)

    for u, egonet in egonets.items():
        partitioning = clustering_fn(egonet)  
        seen_neighbors = set()  
        for partition in partitioning:
            persona_id = next(persona_id_counter)
            persona_to_original_mapping[persona_id] = u
            for v in partition:
                node_neighbor_persona_id_map[u][v] = persona_id
                assert v not in seen_neighbors
                seen_neighbors.add(v)
    for u in graph.nodes(): 
        for v in graph.neighbors(u):
            if v == u:
                continue
            assert v in node_neighbor_persona_id_map[u]
            u_p = node_neighbor_persona_id_map[u][v]
            assert u in node_neighbor_persona_id_map[v]
            v_p = node_neighbor_persona_id_map[v][u]
            persona_graph.add_edge(u_p, v_p)
    return persona_graph, persona_to_original_mapping


In [71]:
def PersonaOverlappingClustering(graph, local_clustering_fn,global_clustering_fn):
    persona_graph, persona_id_mapping = CreatePersonaGraph(graph, local_clustering_fn)
    print(persona_id_mapping)
    print("---")
    print(persona_graph)
    
    non_overlapping_clustering = global_clustering_fn(persona_graph)
    overlapping_clustering = set()
    for cluster in non_overlapping_clustering:
        cluster_original_graph = set([persona_id_mapping[c] for c in cluster])
        cluster_original_graph = list(cluster_original_graph)
        cluster_original_graph.sort()
        overlapping_clustering.add(tuple(cluster_original_graph))
    return list(overlapping_clustering), persona_graph, persona_id_mapping

In [72]:
def main():
    graph = nx.read_edgelist("D:\wsc project\paper_dataset.csv", delimiter="," )
#     nx.draw(graph,with_labels=True)
    print("Select Local Clustering Function ")
    print("\n Select among \n 1.'label_prop'\n 2.Modularity \n 3.connected_components\n") 
    local_func=int(input())   
    print("Select Global Clustering Function ")
    print("\n Select among \n 1.'label_prop'\n 2.'Modularity' \n 3.'connected_components'\n")
    global_func=int(input())
    
    
    local_clustering_fn = CLUSTERING_FN[local_func]
    global_clustering_fn = CLUSTERING_FN[global_func]
    
    
   # min_component_size=int(input("Enter Minimum component size\n"))
    clustering, persona_graph, persona_id_mapping = PersonaOverlappingClustering(graph, local_clustering_fn, global_clustering_fn)
    with open("overlappingClusters.txt", 'w') as outfile:
        for cluster in clustering:
            outfile.write(' '.join([str(x) for x in cluster]) + '\n')
    if persona_graph is not None:
        nx.write_edgelist(persona_graph, "persona-Graph.txt")
    if persona_id_mapping is not None:
        with open("persona-Graph-Id_mapping.txt", 'w') as outfile:
            for persona_node, original_node in persona_id_mapping.items():
                outfile.write('{} {}\n'.format(persona_node, original_node))
        
main()

Select Local Clustering Function 

 Select among 
 1.'label_prop'
 2.Modularity 
 3.connected_components

1
Select Global Clustering Function 

 Select among 
 1.'label_prop'
 2.'Modularity' 
 3.'connected_components'

1
{0: 'b', 1: 'c', 2: 'c', 3: 'a', 4: 'e', 5: 'f', 6: 'f', 7: 'd', 8: 'h', 9: 'g'}
---

